<a href="https://colab.research.google.com/github/ehdob-cid20/trabalho-pratico/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#  Pré Processamento

## Importando as bibliotecas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importando o *dataset*

### Verifica se o dataset já foi importado para o google drive

Um script que verifica se o dataset já foi importado para o google drive. Caso ainda não tenha sido importado, ele faz o download do arquivo e extrai para a pasta informada na variável **dirName**.

Variáveis:

> **datasetFileName** é o nome do arquivo com extensão.

> **datsetUrl** é o endereço web onde o arquivo está disponível.

> **dirName** é o diretório do google drive onde o dataset será salvo.

Nota: O diretório de salvamento (*dirName*) precisa existir.


In [4]:
import os

datasetFileName = 'dados-curso-completo.csv.tar.gz'
datsetUrl = 'http://tiagodemelo.info/datasets/dados-curso-completo.csv.tar.gz'
dirName = 'trabalho-pratico' # Este diretorio pode ser alterado se ja tiver o dataset no google drive.

if False == os.path.isfile('/content/drive/My Drive/' + dirName + '/' + datasetFileName[:-7]):
  if False == os.path.isdir('/content/drive'):
    print('Google driver não está montado!')
  elif False == os.path.isdir('/content/drive/My Drive/' +  dirName + '/'):
    print('Diretório ' + dirName + ' não encontrado.')
  elif False == os.path.isfile('/content/drive/My Drive/' + dirName + '/' + datasetFileName):
    !wget -O '/content/drive/My Drive/$dirName/$datasetFileName' '$datsetUrl' #TODO: handle download errors.
    !tar -C '/content/drive/My Drive/$dirName/{datasetFileName[:-7]}' -zxf '/content/drive/My Drive/$dirName/$datasetFileName' #TODO: handle extract errors.
  else:
    !tar -C '/content/drive/My Drive/$dirName/' -zxf '/content/drive/My Drive/$dirName/$datasetFileName' #TODO: handle extract errors.
else:
  print('O dataset ' + datasetFileName[:-7] + ' foi encontrado.')

O dataset dados-curso-completo.csv foi encontrado.


### Importando o *dataset* usando pandas

In [5]:
dataset = pd.read_csv('/content/drive/My Drive/' + dirName + '/' + datasetFileName[:-7])

## Visualizando o *dataset* inicial (sem qualquer tratamento)



> Segue uma informação geral do dataset tento colunas, contagem e tipo. Em seguida segue uma visão geral de cada campo.



In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658825 entries, 0 to 1658824
Data columns (total 13 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   data        1658825 non-null  object 
 1   usuario     1658768 non-null  object 
 2   apelido     1658825 non-null  object 
 3   texto       1658825 non-null  object 
 4   retweet     1658825 non-null  int64  
 5   seguidores  1658825 non-null  int64  
 6   idioma      1658825 non-null  object 
 7   lugar       74605 non-null    object 
 8   pais        74600 non-null    object 
 9   sigla       74595 non-null    object 
 10  latitude    8720 non-null     float64
 11  longitude   8720 non-null     float64
 12  hashtags    1658825 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 164.5+ MB


### Sumário para o campo *usuario*

**Resumo geral:**
> Possui 1.658.825 campos, onde 632.601 são únicos, 1.026.167 são duplicados e 57 estão nulos.

> Usuário com maior frequência '.',  aparece 1624.

In [19]:
dataset.usuario.describe()

count     1658768
unique     632601
top             .
freq         1624
Name: usuario, dtype: object

In [20]:
usuarios_total = len(dataset.usuario)
usuarios_unicos = dataset.usuario.nunique()
usuarios_null = dataset.usuario.isnull().sum()
usuarios_duplicados = usuarios_total - usuarios_unicos - usuarios_null
usuarios_df = pd.DataFrame({
                  'Usuários': ['únicos', 'duplicados', 'vazio'] ,
                   'Contagem' : [usuarios_unicos, usuarios_duplicados, usuarios_null],
                   'Porcentagem' : [(usuarios_unicos/usuarios_total)*100, (usuarios_duplicados/usuarios_total)*100, (usuarios_null/usuarios_total)*100]})
usuarios_df

,Usuários,Contagem,Porcentagem
0,únicos,632601,38.135487
1,duplicados,1026167,61.861076
2,vazio,57,0.003436


Nota: O campo 'usuario' está trocado com o campo 'apelido', pois usuário é usado como um campo único, enquanto que um apelido pode ser usado por mais de um usuário.

### Sumário para o campo *apelido*

**Resumo geral:**
> Possui 1.658.825 campos, onde 774.516 são únicos e 0 estão nulos.

> Usuário com maior frequência 'Lusa_noticias',  aparece 1404.

In [31]:
dataset.apelido.describe()

count           1658825
unique           774516
top       Lusa_noticias
freq               1404
Name: apelido, dtype: object

In [29]:
apelidos_total = len(dataset.apelido)
apelidos_unicos = dataset.apelido.nunique()
apelidos_null = dataset.apelido.isnull().sum()
apelidos_duplicados = apelidos_total - apelidos_unicos - apelidos_null
apelidos_df = pd.DataFrame({
                  'Apelidos': ['únicos', 'duplicados', 'vazio'] ,
                   'Contagem' : [apelidos_unicos, apelidos_duplicados, apelidos_null],
                   'Porcentagem' : [(apelidos_unicos/apelidos_total)*100, (apelidos_duplicados/apelidos_total)*100, (apelidos_null/apelidos_total)*100]})
apelidos_df

,Apelidos,Contagem,Porcentagem
0,únicos,774516,46.690639
1,duplicados,884309,53.309361
2,vazio,0,0.000000


### Sumário para o campo *texto*

**Resuno geral:**
> Possui 1658825 campos, onde 1653599 são únicos


> Texto com maior frequência "covid 19? desculpa eu entendi convite para ficar de love", frequência do texto 120.

In [ ]:
dataset.texto.describe()

count                                               1658825
unique                                              1653599
top       covid 19? desculpa eu entendi convite pra fica...
freq                                                    120
Name: texto, dtype: object

**Texto duplicados:**


> Os texto que possuem maior duplicidade são:

*   covid 19? desculpa eu entendi convite para ficar de love
*   e vamos de pandemia
* Temos muitos infectados, mas não é de coronavírus, é maldade e inveja...
* e vamos de isolamento social

In [ ]:
duplicado  = dataset.texto.value_counts().reset_index().rename(columns={'index': 'Texto', 'texto': 'Total'})
duplicado

,Texto,Total
0,covid 19? desculpa eu entendi convite pra fica...,120
1,e vamos de pandemia,82
2,"Tem muitos infectados, mas não é de coronavíru...",70
3,e vamos de isolamento social,63
4,#Podcast - Tudo sobre o #Coronavirus aqui. Ouç...,50
...,...,...
1653594,“Decidi doar metade do meu salário de Vereador...,1
1653595,@OsmarTerra INFECTOLOGISTA DIDIER RAOULT QUE P...,1
1653596,RT @birbteef: BMR 41 https://t.co/SKmb7eaD8e,1
1653597,"Coronavírus\nApós cobranças do SINDSALEM, ALEM...",1


### Sumario para o campo *hashtags*

**Resumo geral:**

> Este campo possui 63838 campos únicos, a informação mais frenquente é o "[ ]"



In [ ]:
dataset.hashtags.describe()

count     1658825
unique      63838
top            []
freq      1495569
Name: hashtags, dtype: object

**Hashtags em destaque:**



> Segue abaixo as 10 hashtags mais usadas:



In [ ]:
dataset.hashtags.value_counts()[:10]

[]                 1495569
['COVID19']           9951
['coronavirus']       6658
['Coronavirus']       3294
['coronavírus']       2890
['Covid_19']          2485
['covid19']           2403
['Covid19']           2367
['Coronavírus']       1566
['Repost']             831
Name: hashtags, dtype: int64



> Uma apresentação das hashtags mais frequentes acima e menos frequentes abaixo.



In [ ]:
dataset.hashtags.value_counts()

[]                                                   1495569
['COVID19']                                             9951
['coronavirus']                                         6658
['Coronavirus']                                         3294
['coronavírus']                                         2890
                                                      ...   
['covid19', 'fozdoIguaçu']                                 1
['mininaked', 'naked', 'present']                          1
['ColunadoMafuz', 'clubes', 'jogadores']                   1
['HCNoticias', 'esporte', 'suspenso', 'pandemia']          1
['CoronaDiario']                                           1
Name: hashtags, Length: 63838, dtype: int64

### Sumario para o campo *idioma*

**Resumo geral:**



> No campo idioma temos 59 registros únicos, onde o maior frequente é "pt" com 1586796



In [ ]:
dataset.idioma.describe()

count     1658825
unique         59
top            pt
freq      1586796
Name: idioma, dtype: object



> Contagem de registro por pais



In [ ]:
dataset.idioma.value_counts()

pt     1586796
en       23991
es       14255
it        6480
und       5656
ar        4579
fr        3656
ca        2600
tr        2296
ja        1955
in         948
th         890
tl         604
ko         544
hi         485
ru         417
de         361
pl         262
fa         258
nl         208
ur         179
ht         167
et         162
eu         116
el          94
lt          86
sv          84
cs          72
no          56
zh          51
da          45
ro          45
fi          44
cy          39
uk          37
lv          36
iw          36
ta          32
hu          26
vi          23
is          21
sr          20
sl          18
te          17
sd          12
ne          11
bn          10
mr          10
gu           7
ps           6
ckb          5
bg           4
am           4
or           2
si           2
ml           2
dv           1
kn           1
my           1
Name: idioma, dtype: int64



> Contagem de usuarios por pais



In [ ]:
dataset.groupby('idioma').usuario.count()

idioma
am           4
ar        4578
bg           4
bn          10
ca        2600
ckb          5
cs          72
cy          39
da          45
de         361
dv           1
el          94
en       23991
es       14255
et         162
eu         116
fa         258
fi          44
fr        3655
gu           7
hi         485
ht         167
hu          26
in         948
is          21
it        6480
iw          36
ja        1955
kn           1
ko         544
lt          86
lv          36
ml           2
mr          10
my           1
ne          11
nl         208
no          56
or           2
pl         262
ps           6
pt     1586743
ro          45
ru         417
sd          12
si           2
sl          18
sr          20
sv          84
ta          32
te          17
th         889
tl         604
tr        2296
uk          37
und       5655
ur         179
vi          23
zh          51
Name: usuario, dtype: int64

## Aplicando tratamento ao *dataset*

### Removendo duplicatas

### Categorizando variáveis

## Dividindo o dataset em *Training set* e *Test set*